In [9]:
import math
import matplotlib.pyplot as plt
from matplotlib import style
style.use('fivethirtyeight')
from sklearn.datasets import make_blobs
import numpy as np
from scipy.stats import multivariate_normal

In [10]:
# 0. Create dataset
X,Y = make_blobs(cluster_std=1.5,random_state=20,n_samples=500,centers=3)

In [11]:
# Stretch dataset to get ellipsoid data
X = np.dot(X,np.random.RandomState(0).randn(2,2))

In [20]:
def GMM(X,number_of_sources,iterations):
        reg_cov = 1e-6*np.identity(len(X[0]))
        x,y = np.meshgrid(np.sort(X[:,0]),np.sort(X[:,1]))
        XY = np.array([x.flatten(),y.flatten()]).T
           
        mu = np.zeros((number_of_sources,len(X[0]))) # This is a nxm matrix since we assume n sources (n Gaussians) where each has m dimensions
        cov = np.zeros((number_of_sources,len(X[0]),len(X[0]))) # We need a nxmxm covariance matrix for each source since we have m features --> We create symmetric covariance matrices with ones on the digonal
        for dim in range(len(cov)):
            np.fill_diagonal(cov[dim],5)


        pi = np.ones(number_of_sources)/number_of_sources # Are "Fractions"
        log_likelihoods = [] 
            
        for i in range(iterations):               

            """E Step"""
            global r_ic
            r_ic = np.zeros((len(X),len(cov)))

            for m,co,p,r in zip(mu,cov,pi,range(len(r_ic[0]))):
                co+= reg_cov
                mn = multivariate_normal(mean=m,cov=co)
                r_ic[:,r] = p*mn.pdf(X)/np.sum([pi_c*multivariate_normal(mean=mu_c,cov=cov_c).pdf(X) for pi_c,mu_c,cov_c in zip(pi,mu,cov+reg_cov)],axis=0)


            """M Step"""

            # Calculate the new mean vector and new covariance matrices, based on the probable membership of the single x_i to classes c --> r_ic
            mu = []
            cov = []
            pi = []
            log_likelihood = []

            for c in range(len(r_ic[0])):
                m_c = np.sum(r_ic[:,c],axis=0)
                mu_c = (1/m_c)*np.sum(X*r_ic[:,c].reshape(len(X),1),axis=0)
                mu.append(mu_c)

                # Calculate the covariance matrix per source based on the new mean
                cov.append(((1/m_c)*np.dot((np.array(r_ic[:,c]).reshape(len(X),1)*(X-mu_c)).T,(X-mu_c)))+reg_cov)
                # Calculate pi_new which is the "fraction of points" respectively the fraction of the probability assigned to each source 
                pi.append(m_c/np.sum(r_ic)) 

            
            """Log likelihood"""
            log_likelihoods.append(np.log(np.sum([k*multivariate_normal(mu[i],cov[j]).pdf(X) for k,i,j in zip(pi,range(len(mu)),range(len(cov)))])))    
        print("Max log likelyhood:",log_likelihoods[-1])
        def bic():
            d = number_of_sources*3 - 1
            value = log_likelihoods[-1] - (d/2)*math.log(len(X))
            return value
        print("BIC=",bic())
        print("r_ic:",r_ic)
        

In [21]:
GMM(X,3,100)

Max log likelyhood: 0.5254555602076895
BIC= -24.332976833481077
r_ic: [[2.43991280e-46 7.74661917e-23 9.99999963e-01]
 [1.81827680e-01 8.18171988e-01 1.92350707e-17]
 [6.73809989e-49 2.09937565e-24 9.99999839e-01]
 ...
 [9.97928859e-01 2.07106398e-03 3.08656648e-30]
 [9.81913858e-01 1.80861382e-02 6.17594384e-22]
 [1.04860533e-04 9.99895216e-01 2.82790212e-11]]


In [22]:
r_ic

array([[2.43991280e-46, 7.74661917e-23, 9.99999963e-01],
       [1.81827680e-01, 8.18171988e-01, 1.92350707e-17],
       [6.73809989e-49, 2.09937565e-24, 9.99999839e-01],
       ...,
       [9.97928859e-01, 2.07106398e-03, 3.08656648e-30],
       [9.81913858e-01, 1.80861382e-02, 6.17594384e-22],
       [1.04860533e-04, 9.99895216e-01, 2.82790212e-11]])

In [24]:
len(r_ic[0])

3

In [25]:
np.log?

In [26]:
logric = np.log(r_ic)

In [27]:
logric

array([[-1.05026952e+02, -5.09122006e+01, -3.72040728e-08],
       [-1.70469585e+00, -2.00682710e-01, -3.84897965e+01],
       [-1.10918892e+02, -5.45204022e+01, -1.60544033e-07],
       ...,
       [-2.07328876e-03, -6.17969280e+00, -6.79504935e+01],
       [-1.82516960e-02, -4.01260948e+00, -4.88362103e+01],
       [-9.16287935e+00, -1.04789121e-04, -2.42889009e+01]])

In [31]:
multiplo= np.multiply(r_ic,logric)
len(multiplo), len(multiplo[0])

(500, 3)

In [39]:
sumline = multiplo.sum(axis=1)
len(sumline)

array([-3.72040715e-08, -4.74153864e-01, -1.60544007e-07,  5.51670171e-08,
       -8.46762879e-03, -9.22412242e-03, -2.28619924e-03, -8.23225311e-04,
       -1.43641993e-07,  6.50059475e-10,  1.06684939e-07, -7.26287802e-04,
       -8.36114353e-03, -4.14660412e-03, -4.16802844e-04, -2.38642988e-08,
       -1.06437025e-06,  1.46017942e-07, -7.29127521e-04, -9.72430421e-04,
        8.05599785e-08,  5.32270896e-08, -2.05637381e-03, -1.93301609e-02,
       -1.12878805e-01, -2.56404239e-03,  1.13588929e-07, -5.92399044e-05,
        5.47726293e-08, -8.54103071e-07, -5.94631559e-01, -2.73240318e-01,
       -5.59943367e-03, -1.37933038e-05,  1.39754135e-07, -6.89371742e-01,
        2.74616225e-08, -1.60781142e-03, -6.90366240e-04, -1.68240829e-06,
       -1.32521703e-04, -1.18684008e-04, -1.01683717e-04, -3.31484432e-01,
       -8.23696020e-02, -1.46243396e-04,  1.47355913e-07, -4.25273170e-08,
        1.46214127e-07, -7.42101758e-05, -1.04958922e-03, -7.30463125e-05,
       -2.14302400e-03, -

In [41]:
ENT = sumline.sum()
ENT

-16.876409592107105

In [102]:
import math
n=len(X)
def bic(likelyhood,number_of_sources):
    d= 3*number_of_sources - 1
    return likelyhood - d*math.log(n)

In [105]:
lik = []
BIC = []
n_components_range = range(1,10)
for n_components in n_components_range:
        lik.append(GMM(X,number_of_sources=n_components,iterations=100))
        #BIC.append(bic(likelyhood=GMM(X,number_of_sources=n_components,iterations=100),number_of_sources=n_components))

-0.3419939613380776
0.36993021756128786
0.5254555602076895
0.5395125030818928
0.6069696489268553
0.5423067831604514
0.5522505284469958
0.554328668619232
0.5557360206517902
